# Estimating Queue Priority Wait Times

https://dl.acm.org/doi/10.1145/3427921.3450249

In [1]:
import pandas
from scipy.optimize import newton

In [14]:
P_GROUP_SIZE = 4
P_GROUP_BLOCK_INTERVAL_DICTIONARY = {
    2 : [1],
    4 : [1,3,10],
    6 : [1,2,4,8,23],
    8 : [1,2,4,7,13,23,50]
}
P_GROUP_WAITING_TIMES = {}

COLS_TO_USE = [4,5,6,9,10,12,13,14,15,16]
HEADER_NAMES = ['size','weight','received_time','fee','confirmed_block_height','confirm_time','waiting_time','fee_rate','enter_block_height','no_block_confirm']

### Data Loading

In [3]:
d1 = pandas.read_csv('TimetxinBlock621500.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
d2 = pandas.read_csv('TimetxinBlock622000.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
d3 = pandas.read_csv('TimetxinBlock622500.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
dataFrame = pandas.concat([d1, d2, d3])

dataFrame = dataFrame[dataFrame['waiting_time'] >= 0] # Remove any rows with negative values for waiting_time
print("Transaction Count: ", len(dataFrame))

Transaction Count:  3389521


### Block Size & Service Rate (Mu)

In [15]:
block_groups = dataFrame.groupby(['confirmed_block_height'])['confirmed_block_height'].count()
mean_block_size = float(round(block_groups.mean()))
service_time = 600
mu = 1/service_time # = 0.0016666666666666668
#mu = (1/service_time)*mean_block_size # = 3.776666666666667
print("Mean Block Size: ", mean_block_size)
print("Mu: ", mu)

Mean Block Size:  2266.0
Mu:  0.0016666666666666668


### Priority Group Bounds (Block Intervals)

In [16]:
def calculate_block_intervals(num_groups):
    match num_groups:
        case 2: return [1]
        case 4: return [1,3,10]
        case 6: return [1,2,4,8,23]
        case 8: return [1,2,4,7,13,23,50]
        case _: return [1,3,10]

block_intervals = calculate_block_intervals(P_GROUP_SIZE)
print("Block Intervals:", block_intervals)

Block Intervals: [1, 3, 10]


### Priority Group Range (Fee Rate)

In [17]:
def calculate_feerate_for_priority_groups(data, upper_boundary):
    feerate_range = []
    transaction_counts = 0
    feerate_range.append(data['fee_rate'].max())
    for i in range(len(upper_boundary)):
        transaction_counts = len(data[data['no_block_confirm'] <= upper_boundary[i]])
        #boundary = data['fee_rate'].nlargest(transaction_counts).iloc[-1]
        boundary = data['fee_rate'].tail(transaction_counts).iloc[0]
        feerate_range.append(boundary)
    feerate_range.append(0.0)
    return feerate_range

p_groups_feerate_range = calculate_feerate_for_priority_groups(dataFrame.sort_values('fee_rate'), block_intervals)
print("Priority Group Range (Fee Rate): ", p_groups_feerate_range)

Priority Group Range (Fee Rate):  [52380.95238095238, 30.175438596491237, 18.529411764705888, 5.0, 0.0]


### Arrival Rates (Lambda) - Priority

In [18]:
def calculate_lambda_for_priority_groups(data, feerate_range):
    p_groups_lambda = []
    received_time_max = data['received_time'].max()
    received_time_min = data['received_time'].min()
    for current, next in zip(feerate_range,feerate_range[1:]):
        p_df = data[(data['fee_rate'] <= current) & (data['fee_rate'] > next)]
        p_transaction_count = len(p_df)
        p_lambda = float(p_transaction_count) / (received_time_max - received_time_min)
        p_groups_lambda.append(p_lambda)
    return p_groups_lambda

raw_wait_times = []
p_groups_lambda = []
transaction_count = len(dataFrame)
received_time_max = dataFrame['received_time'].max()
received_time_min = dataFrame['received_time'].min()

for current, next in zip(p_groups_feerate_range,p_groups_feerate_range[1:]):
    p_df = dataFrame[(dataFrame['fee_rate'] <= current) & (dataFrame['fee_rate'] > next)]
    p_transaction_count = len(p_df)
    p_lambda = float(p_transaction_count) / (received_time_max - received_time_min)
    p_groups_lambda.append(p_lambda)
    raw_wait_times.append(p_df['waiting_time'].mean())

    print("Priority Group " + str(p_groups_feerate_range.index(current)+1))
    print("Transaction Count: " + str(p_transaction_count) + ", Raw Wait Time (Mean): " + str(p_df['waiting_time'].mean()))
    print("Feerate Min: " + str(float(p_df['fee_rate'].min())) + ", Max: " + str(float(p_df['fee_rate'].max())))
    print("Lambda: " + str(p_lambda))
    
print("\nSum Of All Lambda: ", sum(p_groups_lambda))

Priority Group 1
Transaction Count: 2035967, Raw Wait Time (Mean): 2474.0209728350214
Feerate Min: 30.175580221997983, Max: 52380.95238095238
Lambda: 1.851985231178986
Priority Group 2
Transaction Count: 565691, Raw Wait Time (Mean): 6085.267361510082
Feerate Min: 18.52950075642965, Max: 30.175438596491237
Lambda: 0.5145718852078014
Priority Group 3
Transaction Count: 385863, Raw Wait Time (Mean): 9987.642969136714
Feerate Min: 5.000028548752705, Max: 18.529411764705888
Lambda: 0.3509941847085032
Priority Group 4
Transaction Count: 402000, Raw Wait Time (Mean): 11979.932554726369
Feerate Min: 0.907563025210084, Max: 5.0
Lambda: 0.36567295193583804

Sum Of All Lambda:  3.083224253031129


### Validation Of Z Using Newton Method (Mine Vs SciPy)

In [19]:
def NewtonMethod(lam, m_u, block_size, x0, epsilon):
    fx = lambda x: lam*(1 - x) - m_u*x*(1 - x**block_size)
    dfx = lambda x: m_u*(block_size*x**block_size + x**block_size - 1) - lam
    while True:
        x1 = x0 - fx(x0) / dfx(x0)
        t = abs(x1 - x0)
        if t < epsilon:
            break
        x0 = x1
    return x0

def SciPyNewton(lam, m_u, block_size, x0, epsilon, max_iteration):
    fx = lambda x: lam*(1 - x) - m_u*x*(1 - x**block_size)
    #dfx = lambda x: m_u*(block_size*x**block_size + x**block_size - 1) - lam
    #return newton(fx, x0, fprime=dfx, args=(), tol=epsilon, maxiter=max_iteration, fprime2=None)
    return newton(fx, x0, fprime=None, args=(), tol=epsilon, maxiter=max_iteration, fprime2=None)

def calculate_z_priority(lambda_priority_group, m_u, block_size):
    my_p_group_z = []
    sciPy_p_group_z = []
    lambda_sum = 0.0
    for i in range(len(lambda_priority_group)):
        lambda_sum += lambda_priority_group[i]
        my_p_group_z.append(NewtonMethod(lambda_sum, m_u, block_size, 0, 1e-10))
        sciPy_p_group_z.append(SciPyNewton(lambda_sum, m_u, block_size, 0, 1e-10, 500))
    #return my_p_group_z
    return sciPy_p_group_z
    
my_p_group_z = []
sciPy_p_group_z = []
lambda_sum = 0.0

for i in range(len(p_groups_lambda)):
    lambda_sum += p_groups_lambda[i]
    my_p_group_z.append(NewtonMethod(lambda_sum, mu, mean_block_size, 0, 1e-10))
    sciPy_p_group_z.append(SciPyNewton(lambda_sum, mu, mean_block_size, 0, 1e-10, 500))

for i in range(len(my_p_group_z)):
    print("Priority Group " + str(i+1))
    print("My Z: " + str(my_p_group_z[i]) + ", SciPy Z: " + str(sciPy_p_group_z[i]))
    

Priority Group 1
My Z: 0.9992742750913941, SciPy Z: 0.9992742751058932
Priority Group 2
My Z: 0.9995497987137135, SciPy Z: 0.9995497987138171
Priority Group 3
My Z: 0.9996918411807911, SciPy Z: 0.9996918412628326
Priority Group 4
My Z: 0.9998145619120352, SciPy Z: 0.9998145619140734


### Wait Times Using Z

In [20]:
def calculate_wait_time_for_priority_groups(p_group_lambda, p_group_z):
    prev_transaction_in_queue = 0.0
    p_group_response_time = []
    for i in range(len(p_group_lambda)):
        temp = p_group_z[i] / (1 - p_group_z[i]) # L(y) = z(x) / (1 - z(x))
        transaction_in_queue = temp - prev_transaction_in_queue # L(x) = L(y) - L(y-1)
        prev_transaction_in_queue = temp
        response_time = transaction_in_queue / p_group_lambda[i] # W(x) = L(x) / Lambda(x)
        p_group_response_time.append(response_time)
    return p_group_response_time

prev_transaction_in_queue = 0.0
p_group_response_time = []

for i in range(len(p_groups_lambda)):
    temp = sciPy_p_group_z[i] / (1 - sciPy_p_group_z[i]) # L(y) = z(x) / (1 - z(x))
    transaction_in_queue = temp - prev_transaction_in_queue # L(x) = L(y) - L(y-1)
    prev_transaction_in_queue = temp

    response_time = transaction_in_queue / p_groups_lambda[i] # W(x) = L(x) / Lambda(x)

    p_group_response_time.append(response_time)

    print("Priority Group " + str(i+1))
    print("Average Transaction In Queue (L): ", transaction_in_queue)
    print("Average Response Time In Queue (W): " + str(response_time))


Priority Group 1
Average Transaction In Queue (L):  1376.9326134743274
Average Response Time In Queue (W): 743.4900615259027
Priority Group 2
Average Transaction In Queue (L):  843.2960473290768
Average Response Time In Queue (W): 1638.8303977947135
Priority Group 3
Average Transaction In Queue (L):  1023.8521348672389
Average Response Time In Queue (W): 2917.0059775136638
Priority Group 4
Average Transaction In Queue (L):  2147.554676179624
Average Response Time In Queue (W): 5872.883583023225


In [21]:
def initialise_values(data, block_intervals):
    p_group_wait_time = {}
    block_group = data.groupby(['confirmed_block_height'])['confirmed_block_height'].count()
    average_block_size = float(round(block_group.mean()))
    s_time = 600
    m_u = 1/s_time # = 0.0016666666666666668
    for key, value in block_intervals.items():
        feerate_range = calculate_feerate_for_priority_groups(data.sort_values('fee_rate'), value)
        p_lambda = calculate_lambda_for_priority_groups(data, feerate_range)
        p_z = calculate_z_priority(p_lambda, m_u, average_block_size)
        p_wait_time = calculate_wait_time_for_priority_groups(p_lambda, p_z)
        p_group_wait_time[key] = p_wait_time
    return p_group_wait_time

def Get_Wait_Times_Based_On_Num_Of_Priorities(num_of_priorities):
    return P_GROUP_WAITING_TIMES.get(num_of_priorities)

P_GROUP_WAITING_TIMES = initialise_values(dataFrame, P_GROUP_BLOCK_INTERVAL_DICTIONARY)
print("Wait Times For 2 Priority Groups:", Get_Wait_Times_Based_On_Num_Of_Priorities(4))

Wait Times For 2 Priority Groups: [743.4900615259027, 1638.8303977947135, 2917.0059775136638, 5872.883583023225]
